In [ ]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from yellowbrick.cluster import KElbowVisualizer
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from pyclustering.cluster.cure import cure
from pyclustering.cluster import cluster_visualizer

from pyclustering.samples.definitions import SIMPLE_SAMPLES
from pyclustering.samples.definitions import FCPS_SAMPLES

from pyclustering.utils import read_sample
from pyclustering.utils import timedcall
from sklearn.datasets import make_blobs
from mpl_toolkits import mplot3d
from sklearn import metrics



## 1. Preprocessing 

In [ ]:
#1.1 Khởi tạo dataframe

#path = "..\data\diabetic_data.csv"
path = "..\data\data.csv"
df = pd.read_csv(path)
df.describe()


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
count,58.000000,58.0,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000
mean,1.603448,3.0,14465.431034,8203.000000,10509.568966,2132.534483,4026.120690,2208.551724
std,0.493454,0.0,12424.034595,9365.404286,9167.376757,2438.761364,4332.285022,2664.142712
min,1.000000,3.0,491.000000,555.000000,902.000000,33.000000,100.000000,46.000000
25%,1.000000,3.0,5302.250000,3098.500000,4542.750000,496.750000,999.750000,551.000000
50%,2.000000,3.0,10551.500000,5406.500000,7639.500000,1179.500000,2720.500000,1335.500000
75%,2.000000,3.0,21528.250000,9801.500000,12388.000000,2571.000000,5488.500000,2887.500000
max,2.000000,3.0,56159.000000,54259.000000,55571.000000,10002.000000,24171.000000,16523.000000


In [ ]:
# 1.2 EDA


In [ ]:
# 1.3 Null cleaning

(58, 10)

In [21]:
# 1.4 Outliers cleaning

In [ ]:
# 1.5 Chuẩn hóa dữ liệu và giảm chiều dữ liệu

## 2. CURE Algorithm 

In [ ]:
# 2.1 Chạy giải thuật, huấn luyện mô hình và trực quan hóa dữ liệu
def 

## 3. Evaluation

## 4. Main 